In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_fire_hasoc_tamil_train = pd.read_excel("../data/tamil_hasoc/Tamil-Codemixed_offensive_data_Training-Tweet-HL.xlsx",index_col=0)
df_fire_hasoc_tamil_train = df_fire_hasoc_tamil_train.dropna()

print(df_fire_hasoc_tamil_train.info())

fire_hasoc_tamil_train_sentences = df_fire_hasoc_tamil_train.values[:,0]
fire_hasoc_tamil_train_labels = df_fire_hasoc_tamil_train.values[:,1]

new_fire_hasoc_tamil_train_labels=[x.upper() for x in fire_hasoc_tamil_train_labels]

fire_hasoc_tamil_train_labels = new_fire_hasoc_tamil_train_labels

print(set(fire_hasoc_tamil_train_labels))

df_fire_hasoc_tamil_test = pd.read_csv("../data/tamil_hasoc/Tamil_hasoc_tanglish_test_withlabels.tsv",sep="\t")
df_fire_hasoc_tamil_test = df_fire_hasoc_tamil_test.dropna()

print(df_fire_hasoc_tamil_test.info())
fire_hasoc_tamil_test_sentences = df_fire_hasoc_tamil_test.values[:,1]
fire_hasoc_tamil_test_labels = df_fire_hasoc_tamil_test.values[:,2]

print(set(fire_hasoc_tamil_test_labels))

x_train,x_dev,y_train,y_dev = train_test_split(fire_hasoc_tamil_train_sentences,fire_hasoc_tamil_train_labels,test_size=0.25, random_state=42)

x_test = fire_hasoc_tamil_test_sentences
y_test = fire_hasoc_tamil_test_labels

print(len(x_train),len(x_dev),len(x_test)," total = ",len(x_train)+len(x_dev)+len(x_test))

total_sentences = []
total_sentences.extend(fire_hasoc_tamil_train_sentences)
total_sentences.extend(fire_hasoc_tamil_test_sentences)

total_labels = []
total_labels.extend(fire_hasoc_tamil_train_labels)
total_labels.extend(fire_hasoc_tamil_test_labels)

print(len(total_sentences),len(total_labels))

<class 'pandas.core.frame.DataFrame'>
Index: 3999 entries, TA_HL101 to TA_TW4528
Data columns (total 2 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   Iyaooo Kovam pattutene sothula visatha vachuruvanugale ..iooo...  3999 non-null   object
 1   NOT                                                               3999 non-null   object
dtypes: object(2)
memory usage: 93.7+ KB
None
{'NOT', 'OFF'}
<class 'pandas.core.frame.DataFrame'>
Int64Index: 939 entries, 0 to 938
Data columns (total 3 columns):
 #   Column                                                                                     Non-Null Count  Dtype 
---  ------                                                                                     --------------  ----- 
 0   TA_TW15946                                                                                 939 non-null

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total_sentences)
print(X.shape)

(4938, 19598)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_sim = cosine_similarity(X,X)

In [5]:
sum_rows_cosine_sim = np.sum(cosine_sim,axis=1)

In [6]:
dict_cosine_sim = {}
for i in range(len(y_train)+len(y_dev),sum_rows_cosine_sim.shape[0]):
    dict_cosine_sim[i] = sum_rows_cosine_sim[i]

In [7]:
sorted_dict = dict(sorted(dict_cosine_sim.items(), key=lambda item: item[1],reverse=True))

In [8]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(200, sorted_dict.items())

In [11]:
dict_key_dissmilar={}

for key,value in n_items:
    similar_values = cosine_sim[key]
    count_dissimilar_labels = 0
    count_similar_labels=0
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label == total_labels[i]:
                count_similar_labels+=1
            else:
                count_dissimilar_labels+=1
    diff = count_dissimilar_labels-count_similar_labels
    dict_key_dissmilar[key] = diff


In [12]:
sorted_key_dict = dict(sorted(dict_key_dissmilar.items(), key=lambda item: item[1],reverse=True))
print(sorted_key_dict)

{4824: 223, 4670: 177, 4774: 168, 4707: 167, 4799: 87, 4550: 87, 4205: 87, 4634: 77, 4821: 29, 4859: 29, 4854: 25, 4835: 23, 4836: 14, 4782: 10, 4899: 4, 4840: 0, 4890: -4, 4467: -14, 4740: -20, 4726: -29, 4885: -31, 4925: -34, 4704: -34, 4884: -35, 4905: -45, 4908: -49, 4043: -54, 4455: -57, 4839: -58, 4616: -72, 4842: -73, 4522: -76, 4601: -79, 4149: -84, 4331: -86, 4880: -93, 4690: -93, 4841: -96, 4900: -102, 4849: -110, 4789: -110, 4904: -126, 4737: -130, 4370: -131, 4259: -131, 4875: -135, 4573: -137, 4258: -138, 4850: -140, 4510: -150, 4911: -151, 4620: -151, 4354: -151, 4400: -151, 4858: -152, 4775: -153, 4903: -155, 4249: -158, 4709: -162, 4681: -164, 4348: -169, 4306: -172, 4753: -174, 4853: -174, 4914: -176, 4068: -176, 4815: -176, 4894: -179, 4877: -186, 4428: -187, 4769: -191, 4920: -195, 4192: -195, 4834: -195, 4645: -196, 4760: -196, 4384: -196, 4257: -199, 4830: -204, 4758: -207, 4916: -207, 4343: -208, 4751: -210, 4865: -215, 4888: -216, 4863: -217, 4763: -222, 4867: -2

In [13]:
n_key_items = take(50, sorted_key_dict.items())

In [14]:
from collections import Counter
dict_key_dissmilar_label_count={}
for key,value in n_key_items:
    dissimilar_labels=[]
    similar_values = cosine_sim[key]
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label != total_labels[i]:
                dissimilar_labels.append(total_labels[i])
    counted = Counter(dissimilar_labels)
    most_common_counts = counted.most_common()
    dict_key_dissmilar_label_count[key] = most_common_counts

In [15]:
for key,value in dict_key_dissmilar_label_count.items():
    print(key,value)

4824 [('NOT', 1423)]
4670 [('NOT', 977)]
4774 [('NOT', 935)]
4707 [('NOT', 1332)]
4799 [('NOT', 866)]
4550 [('NOT', 859)]
4205 [('NOT', 736)]
4634 [('NOT', 1238)]
4821 [('OFF', 1134)]
4859 [('NOT', 1045)]
4854 [('NOT', 1316)]
4835 [('OFF', 1016)]
4836 [('OFF', 1128)]
4782 [('OFF', 1006)]
4899 [('NOT', 971)]
4840 [('NOT', 1326)]
4890 [('NOT', 1441)]
4467 [('NOT', 1103)]
4740 [('NOT', 1103)]
4726 [('NOT', 1208)]
4885 [('NOT', 970)]
4925 [('OFF', 1039)]
4704 [('OFF', 1231)]
4884 [('NOT', 1070)]
4905 [('NOT', 1462)]
4908 [('NOT', 1332)]
4043 [('NOT', 1102)]
4455 [('NOT', 1326)]
4839 [('OFF', 1123)]
4616 [('NOT', 1275)]
4842 [('OFF', 1162)]
4522 [('NOT', 902)]
4601 [('NOT', 1221)]
4149 [('NOT', 885)]
4331 [('NOT', 971)]
4880 [('NOT', 1136)]
4690 [('NOT', 1142)]
4841 [('OFF', 1121)]
4900 [('NOT', 1333)]
4849 [('NOT', 1053)]
4789 [('OFF', 954)]
4904 [('NOT', 1105)]
4737 [('OFF', 825)]
4370 [('NOT', 1101)]
4259 [('NOT', 1257)]
4875 [('NOT', 1184)]
4573 [('OFF', 532)]
4258 [('NOT', 1328)]
4850 

In [16]:
qa_data = []
qa_labels=[]
for key,value in n_key_items:
    if value>0:
        qa_data.append(total_sentences[key])
        qa_labels.append(total_labels[key])

In [17]:
qa_df = pd.DataFrame(zip(qa_data,qa_labels),columns=['qa_data','qa_labels'])
qa_df.to_csv('tamil_hasoc_qa.tsv',sep='\t',index=None)